In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath, splitext
from pathlib import Path
from functools import partial
from itertools import product
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which sis why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('inspect_report_corr.ipynb', 'recon' +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dask import delayed, compute
from numba import jit, vectorize, float64, uint
from sortedcontainers import SortedList, SortedSet 

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option('display.max_rows', 500)
# pd.set_option('display.height', 500)
pd.set_option('display.max_columns', 50)

from common_util import DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ, DT_BIZ_DAILY_FREQ, list_get_dict, inner_join, right_join, get_custom_biz_freq_df, query_df, search_df, chained_filter, recursive_dict, benchmark
from common_util import RECON_DIR, load_df, load_json, abs_df, df_count, df_value_count, outer_join, left_join, getcon, string_df_join_to_ser, is_df, is_ser, count_nn_nz_df, pairwise, cust_count, get_pardir_from_path
from data.data_api import DataAPI
from data.access_util import df_getters as dg, col_subsetters2 as cs2
from recon.common import REPORT_DIR, DATASET_DIR, default_gta_dataset, default_gta_test

TODO
- Need to modify gta to use runt based labels

## Library Functions

### Index Value Lookup

In [2]:
def value_lookup(lookup_obj):
    return {
        is_df(lookup_obj): lambda ser: ser.map(lambda idx: lookup_obj.loc[idx, ser.name]),
        is_ser(lookup_obj): lambda ser: ser.map(lambda idx: lookup_obj.loc[idx]),
        None: None
    }.get(True, None)

def index_lookup_dfs(idx_df, *lookup_tables):
    """
    Return index dataframe mapped to values in series or dataframe lookup tables.
    """
    yield from (idx_df.apply(value_lookup(lookup_table), axis=0) for lookup_table in lookup_tables)

### Filtering

In [3]:
def top_n(ser, n=10, sort_type='max', map_index=True):
    top_ser = {
        'min': lambda: ser.nsmallest(n=n),
        'max': lambda: ser.nlargest(n=n)
    }.get(sort_type)()

    if (map_index):
        return top_ser.index
    else:
        return top_ser.values

def get_filter_func(thresh, filter_type='gte'):
    return {
        'gte': (lambda val: val >= thresh if (val is not None and isinstance(val, float)) else False),
        'lte': (lambda val: val <= thresh if (val is not None and isinstance(val, float)) else False)
    }.get(filter_type)

### Report Viewing

In [4]:
def global_top(df, *value_dfs, n=10, sort_type='max'):
    """
    View a filtered representation of df and corresponding value dfs based on sorting method.
    
    Args:
        df (pd.DataFrame): dataframe to apply the sorting to
        *value_dfs (pd.Dataframe): dataframes to index the result of the original df into
        n (int): the top n rows returned per dataframe
        sort_type ('max'|'min'): sorting method
        
    Returns:
        dataframe of column names and values of top n according to sorting method, along with value_dfs
        indexed by the top n indices per column
    """
    top10_idx = df.iloc[:, 2:].apply(partial(top_n, n=n, sort_type=sort_type), axis=0)
    global_max_dfs = index_lookup_dfs(top10_idx, string_df_join_to_ser(df.iloc[:, :2]), df, *value_dfs)
    return global_max_dfs

## Initializing

### Corr Report Config Defaults

In [5]:
dataset_name = default_gta_dataset
test_name = default_gta_test
ret_types = ['pba_oc_return', 'pba_oa_return']
assets = ['sp_500', 'russell_2000', 'nasdaq_100', 'dow_jones']

### Test Defaults

In [6]:
count_tests = ['int_count', 'ratio_count']
corr_tests = ['pearson', 'spearman', 'kendall']
tests = {
    'count': count_tests,
    'corr': corr_tests
}

### *Change Config

In [74]:
dataset_name = 'drl'

### Load Metadata

In [75]:
reports_dir = sep.join([REPORT_DIR +splitext(dataset_name)[0]]) +sep
dataset_dict = load_json(dataset_name, dir_path=DATASET_DIR)

## Viewing

### Load Test Results

In [76]:
test_files = recursive_dict()

for asset in assets:
    asset_dir = sep.join([reports_dir, asset]) +sep
    files = os.listdir(asset_dir)

    for test, variants in tests.items():
        for ret_type, variant in product(ret_types, variants):
            file = getcon(getcon(files, ret_type), variant)
            test_files[asset][ret_type][test][variant] = load_df(file, dir_path=asset_dir)

### View Test Results

### Presets

In [153]:
asset_name = 'sp_500'
ret_type = 'pba_oc_return'
ratio_count_thresh = .9

In [154]:
ic = test_files[asset_name][ret_type]['count']['int_count']
rc = test_files[asset_name][ret_type]['count']['ratio_count']

prn = test_files[asset_name][ret_type]['corr']['pearson']
spr = test_files[asset_name][ret_type]['corr']['spearman']
ken = test_files[asset_name][ret_type]['corr']['kendall']

### View Test Results

In [155]:
count_thresh = .09
abs_corr_df = abs_df(spr)
filtered_df = abs_corr_df[rc >= count_thresh]

In [156]:
eod_qd = {
    'exact': ['feat_df_desc', 'feat_col_name'],
    'startswith': [],
    'endswith': ['_eod(0%)', '_eod(1%)', '_eod(2%)'],
    'regex': [],
    'exclude': None
}

In [157]:
# filtered_df = filtered_df.loc[:, chained_filter(filtered_df, eod_qd)]
# rc = rc.loc[:, chained_filter(rc, eod_qd)]
# ic = ic.loc[:, chained_filter(ic, eod_qd)]

In [158]:
display(*global_top(filtered_df, rc, ic, sort_type='max', n=10))

,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mag,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mom,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_vel,pba_oc_return_fth_af_abs_avg_shf_1_dir_conf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fb,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_1_dir_mag,pba_oc_return_fth_af_abs_avg_shf_1_dir_mom,pba_oc_return_fth_af_abs_avg_shf_1_dir_vel,pba_oc_return_fth_af_abs_avg_shf_2_dir_conf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fb,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_2_dir_mag,pba_oc_return_fth_af_abs_avg_shf_2_dir_mom,pba_oc_return_fth_af_abs_avg_shf_2_dir_vel,pba_oc_return_fth_af_abs_max_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbeod,...,vol_oc_return_fth_of_abs_xmax_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mag,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mom,vol_oc_return_fth_of_abs_xmax_shf_2_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_1_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_1_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_2_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_2_dir_vel
0,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceDirection_N,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,...,thresh_pba_pba_oc_return_fth_of_xact,raw_trmi_v2_etf_v2_conflict,raw_trmi_v2_etf_v2_conflict,raw_trmi_v2_etf_v2_conflict,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,thresh_pba_pba_oc_return_fth_of_xact,raw_trmi_v3_etf_v3_litigation,raw_trmi_v3_etf_v3_litigation,raw_trmi_v3_etf_v3_litigation,thresh_pba_pba_oc_return_fth_of_xact,raw_trmi_v3_etf_v

,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mag,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mom,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_vel,pba_oc_return_fth_af_abs_avg_shf_1_dir_conf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fb,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_1_dir_mag,pba_oc_return_fth_af_abs_avg_shf_1_dir_mom,pba_oc_return_fth_af_abs_avg_shf_1_dir_vel,pba_oc_return_fth_af_abs_avg_shf_2_dir_conf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fb,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_2_dir_mag,pba_oc_return_fth_af_abs_avg_shf_2_dir_mom,pba_oc_return_fth_af_abs_avg_shf_2_dir_vel,pba_oc_return_fth_af_abs_max_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbeod,...,vol_oc_return_fth_of_abs_xmax_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mag,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mom,vol_oc_return_fth_of_abs_xmax_shf_2_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_1_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_1_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_2_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_2_dir_vel
0,0.072519,0.090629,0.089727,0.090084,0.085641,0.091894,0.092825,0.073190,0.078189,0.080355,0.079007,0.081459,0.086501,0.086643,0.065664,0.069571,0.069176,0.062722,0.070776,0.073625,0.072230,0.077689,0.075031,0.080151,0.069183,...,0.055062,0.047071,0.047021,0.047358,0.056950,0.064690,0.061905,0.059310,0.068865,0.071304,0.066772,0.056645,0.058901,0.058090,0.062300,0.060725,0.062169,0.060001,0.052374,0.050156,0.051419,0.064740,0.052309,0.051727,0.050115
1,0.071974,0.090448,0.087148,0.089892,0.082629,0.089049,0.090489,0.071138,0.077670,0.078974,0.078531,0.078627,0.084162,0.084912,0.061768,0.066324,0.065525,0.059726,0.066910,0.069835,0.068687,0.074314,0.073920,0.077800,0.068467,...,0.053799,0.046656,0.046607,0.047148,0.053678,0.060436,0.058071,0.055915,0.064803,0.066837,0.062183,0.051689,0.053425,0.052849,0.058793,0.055466,0.056559,0.054216,0.051384,0.049822,0.050755,0.062126,0.051294,0.051181,0.050091
2,0.067714,0.090248,0.083959,0.089859,0.057377,0.074478,0.088148,0.057534,0.074580,0.070371,0.076230,0.054386,0.061799,0.079512,0.055800,0.054187,0.056384,0.047541,0.053968,0.057644,0.066910,0.048703,0.065437,0.060025,0.061589,...,0.044520,0.043424,0.043419,0.043379,0.045944,0.048581,0.048210,0.051599,0.047845,0.047128,0.048382,0.042105,0.044067,0.042648,0.050475,0.044864,0.045860,0.045190,0.048777,0.048355,0.048718,0.049306,0.048907,0.048823,0.048171
3,0.066122,0.089002,0.081855,0.089192,0.055439,0.067903,0.087897,0.053242,0.067253,0.063562,0.069477,0.052753,0.054580,0.073617,0.049412,0.050579,0.048245,0.046342,0.052285,0.050551,0.063056,0.046384,0.053580,0.049388,0.050348,...,0.043273,0.041649,0.041872,0.042345,0.045380,0.048381,0.047199,0.046298,0.047562,0.046199,0.048344,0.040763,0.042565,0.042180,0.047890,0.044147,0.04

,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mag,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mom,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_vel,pba_oc_return_fth_af_abs_avg_shf_1_dir_conf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fb,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_1_dir_mag,pba_oc_return_fth_af_abs_avg_shf_1_dir_mom,pba_oc_return_fth_af_abs_avg_shf_1_dir_vel,pba_oc_return_fth_af_abs_avg_shf_2_dir_conf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fb,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_2_dir_mag,pba_oc_return_fth_af_abs_avg_shf_2_dir_mom,pba_oc_return_fth_af_abs_avg_shf_2_dir_vel,pba_oc_return_fth_af_abs_max_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbeod,...,vol_oc_return_fth_of_abs_xmax_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mag,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mom,vol_oc_return_fth_of_abs_xmax_shf_2_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_1_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_1_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_2_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_2_dir_vel
0,0.997406,0.997406,0.997406,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.00000,1.00000,1.000000,1.0000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.00000,1.00000,1.000000,1.0000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,1.000000,0.997406,0.997406,0.997406,0.997406,0.997406,0.997406,0.997406,0.997406,1.000000,0.997406,0.997406,0.999800,0.999800,0.999800,1.000000,0.999800,0.999800,0.997406,0.997406,0.997406,0.997406,0.997406,...,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,0.916250,1.000000,1.0,0.91625,1.00000,1.000000,1.0000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.999800,1.000000,1.000000,0.999800,0.997406,0.999800,0.997406,0.999800,0.997406,0.999800,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.0,1.0,1.0,0.997402,1.000000,1.000000,1.000000,1.000000,0.999800,1.0,1.00000,1.00000,0.916250,1.0000,0.916250,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,1.000000,1.000000,0

,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mag,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mom,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_vel,pba_oc_return_fth_af_abs_avg_shf_1_dir_conf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fb,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_1_dir_mag,pba_oc_return_fth_af_abs_avg_shf_1_dir_mom,pba_oc_return_fth_af_abs_avg_shf_1_dir_vel,pba_oc_return_fth_af_abs_avg_shf_2_dir_conf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fb,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_2_dir_mag,pba_oc_return_fth_af_abs_avg_shf_2_dir_mom,pba_oc_return_fth_af_abs_avg_shf_2_dir_vel,pba_oc_return_fth_af_abs_max_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbeod,...,vol_oc_return_fth_of_abs_xmax_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mag,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mom,vol_oc_return_fth_of_abs_xmax_shf_2_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_1_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_1_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_2_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_2_dir_vel
0,5000.0,5000.0,5000.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,...,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0
1,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,...,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0
2,5013.0,5013.0,5013.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5013.0,5000.0,5000.0,5012.0,5012.0,5012.0,5013.0,5012.0,5012.0,5000.0,5000.0,5000.0,5000.0,5000.0,...,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,4590.0,5013.0,5013.0,4590.0,5012.0,5012.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0
3,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5012.0,5013.0,5013.0,5012.0,5000.0,5012.0,5000.0,5012.0,5000.0,5012.0,5013.0,5013.0,5013.0,5013.0,...,5013.0,5013.0,5013.0,5013.0,5000.0,5013.0,5013.0,5013.0,5013.0,5012.0,5013.0,5012.0,5013.0,4590.0,5013.0,4590.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0
4,5013.0,5013.0,5013.0,5013.0,5012.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,4880.0,5013.0,5013.0,5013.0,5012.0,5013.0,5013.0,4880.0,5012.0,5013.0,5012.0,5013.0,5013.0,5013.0,...,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5013.0,5000.0,5013.0,4590.0,5013.0,5013.0,5013.0,5013.0,5012.0,5013.0,5013.0,5013.0,5013.0,5013.0,50